In [ ]:
# （0）json简介：
'''________________________ 1_漂亮的打印出JSON ________________________'''
# import json
# data={"status": "OK", "count": 2, "results": 
#       [{"age": 27, "name": "Oz", "lactose_intolerant": "true"}, 
#       {"age": 29, "name": "Joe", "lactose_intolerant": "false"}]}
# print(json.dumps(data,indent=2))
'''________________________ 2 pandas 使嵌套 JSON 秒变 Dataframe ________________________'''
# 只深入到嵌套第二级
# pd.json_normalize(results, record_path="issues", max_level = 2)

'''________________________ 3 Json 字典 格式化与反格式化 ________________________'''

import json

# 字典
data={"err_msg":"OK","err_no":0,"log_id":2100120274,"result":"好",
        "sn":"1f89c138-2525-11ec-a719-38eaa7ddb52b_ws_0","type":"MID_TEXT"}
print(data["result"])

# 格式化
js=json.dumps(data,indent=2)
print(js)

# 反格式化
try:
    getjs=json.loads(js)
    print(getjs["type"])
    print(getjs["aa"])
except:
    pass
    


In [ ]:
# （一）requests + json 网易`严选文胸评论：—— https://you.163.com/xhr/search/search.json
import requests
import time
import json
# 获取产品列表
def search_keyword(keyword):
    uri = 'https://you.163.com/xhr/search/search.json'
    query = {"keyword": keyword ,"page": 1,"size":10}
    try:
        res = requests.get(uri, params=query).json()
        result = res['data']['directly']['searcherResult']['result']
        product_id = []
        for r in result:
            product_id.append(r['id'])
        return product_id
    except:
        raise
# # 获取评论
def details(product_id):
    url = 'https://you.163.com/xhr/comment/listByItemByTag.json'
    try:
        C_list = []
        for i in range(1, 2):
            query = {"itemId": product_id,"page": i,}
            res = requests.get(url, params=query).json()
            if not res['data']['commentList']:
                break
            print("爬取第 %s 页评论" % i)
            commentList = res['data']['commentList']
            C_list.extend(commentList)
            time.sleep(1)
        return C_list
    except:
        raise
        
product_id = search_keyword('文胸')
print(product_id)

r_list = []
for p in product_id:
    r_list.extend(details(p))
with open('./comments.txt', 'w') as f:
    for r in r_list:
        try:
            f.write(json.dumps(r, ensure_ascii=False) + '\n')
        except:
            print('出错啦')

In [ ]:
# （二）json 结构与美化输出：网易`严选文胸 —— https://you.163.com/xhr/comment/listByItemByTag.json?
import json
import requests
get_comment=[]
url_1="https://you.163.com/xhr/comment/listByItemByTag.json?__timestamp=1601645976756&itemId=3987228&tag=%E5%85%A8%E9%83%A8&size=20&page=1&orderBy=0&oldItemTag=%E5%85%A8%E9%83%A8&oldItemOrderBy=0&tagChanged=0"
res=requests.get(url_1).json()
print(res)
commentList = res['data']['commentList']
get_comment.extend(commentList)
# print(json.dumps(commentList,ensure_ascii=False,indent=1))

# url_2="https://you.163.com/xhr/comment/listByItemByTag.json?__timestamp=1601645976756&itemId=3987228&tag=%E5%85%A8%E9%83%A8&size=20&page=2&orderBy=0&oldItemTag=%E5%85%A8%E9%83%A8&oldItemOrderBy=0&tagChanged=0"
# res=requests.get(url_2).json()
# commentList = res['data']['commentList']
# get_comment.extend(commentList)
# with open('./commentList.txt','w') as f:
#     for cc in commentList:
#         f.write(json.dumps(cc,ensure_ascii=False)+'\n')

In [ ]:
# （三）requests + base64 + json + cv2 + numpy + matplotlib 调用AI开放平台：以旷视科技的人脸技术为例
'''
腾讯 AI 开放平台：https://ai.qq.com/
百度 AI 开放平台：https://ai.baidu.com/
京东 AI 开放平台：http://neuhub.jd.com/
旷视 AI 开放平台：https://www.faceplusplus.com.cn/
常见的图像技术、语音技术、文字识别，这些网站都有提供。
单看服务数量，百度提供的接口最丰富。
旷视，人脸相关的技术，应该是比较好，毕竟早些年支付宝的人脸识别技术，都是旷视提供技术支持的。
AI 开放平台提供了丰富的 AI 领域的基础能力，怎么用，用来干什么，就看自己的想象力了。
当然，这些免费使用的 API 接口有很多限制，比如不能请求太快等等。
想要 API 提供性能更好，QPS 更大的优质服务，那就得充钱了。

以旷视科技的人脸技术为例进行测试：
原文教程网址：https://mp.weixin.qq.com/s?__biz=MzIxODg1OTk1MA==&mid=2247485136&idx=1&sn=64931d9c8e6e576b1199c8787352b8a6&
chksm=97e55611a092df07eb1d9ddfa68a69119fb21f1a7611a402dac9af62ab1d6ffcc8c7ae5dbcf5&scene=178&cur_album_id=1350803219538149376#rd
图片地址：
https://cuijiahua.com/wp-content/uploads/2020/05/test_1.png
（1）首先，创建一个账号。
（2）然后，找一个想体验的服务，咱先试试美颜美型：https://www.faceplusplus.com.cn/face-beautify/
（3）登录账号，选择控制台，然后创建一个应用，选择「试用」类型。
（4）填写一些信息后，就会生成一个 API Key 和 API Secret，这两个是使用 API 接口要用到的参数。
    相当于你的个人账号和密码。
（5）在控制台，有各种服务的使用说明，比如人脸美颜。
    Beautify API v2：https://console.faceplusplus.com.cn/documents/134252584
    这个文档，有详细的接口描述，根据这些描述，就可以写代码。
'''
%matplotlib notebook
%matplotlib inline
import requests
import base64
import json
import cv2
import numpy as np
import matplotlib.pyplot as plt
beautify_url = "https://api-cn.faceplusplus.com/facepp/v2/beautify"
# 你创建的应用的 API Key 和 API Secret(也叫 Secret Key)
AK = 'p4E88XpxMKRdNA_sS5ld3TWoGKBwMpK_'
SK = 'vZ4-CHoJSGRxtxiFTabh_ZJuJlW_NtST'

# 可选参数，不填写，默认50
whitening = 80    # 美白程度 0 - 100
smoothing = 90    # 磨皮程度 0 - 100
thinface = 20     # 瘦脸程度 0 - 100
shrink_face = 50  # 小脸程度 0 - 100
enlarge_eye = 50  # 大眼程度 0 - 100
remove_eyebrow = 50  # 去眉毛程度 0 - 100
filter_type = ''     # 滤镜名称，不填写，默认无滤镜

# 二进制方式打开图片，# 必须是英文路径
img_name = 'C:\\Users\\hp\\Desktop\\Face Beautify API.png'
f = open(img_name, 'rb')
# 转 base64
img_base64 = base64.b64encode(f.read())

# 使用 whitening、smoothing、thinface 三个可选参数，其他用默认值
data = {'api_key': AK,
    'api_secret': SK,
    'image_base64': img_base64,  # 传入需要优化的图片：转 base64
    'whitening': whitening,
    'smoothing': smoothing,
    'thinface': thinface,}
r = requests.post(url=beautify_url, data=data)
html = json.loads(r.text)        # API 接口返回的数据是 base64 的二进制文件

# 解析base64图片
base64_data = html['result']
imgData = base64.b64decode(base64_data)
nparr = np.frombuffer(imgData, np.uint8) # uint8是无符号八位整型，表示范围是[0, 255]的整数
img_res = cv2.imdecode(nparr, cv2.IMREAD_COLOR)
img_res_BGR = cv2.cvtColor(img_res, cv2.COLOR_RGB2BGR)

# 原始图片
img = cv2.imread(img_name)   # 必须是英文路径
img_BGR = cv2.cvtColor(img, cv2.COLOR_RGB2BGR)

# 显示图片
fig, axs = plt.subplots(nrows=1, ncols=2, sharex=False, sharey=False, figsize=(10,10))
axs[0].imshow(img_BGR)
axs[1].imshow(img_res_BGR)

plt.show()

In [ ]:
# （四）base64 + json + cv2 + numpy + matplotlib : 写入并展示图片
import base64
import json
import cv2
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

img_name ='C:\\Users\\hp\\Desktop\\Face Beautify API.png'
'''__________________ 方法1：____________________'''
# f = open(img_name, 'rb')
# nparr = np.frombuffer(f.read(), np.uint8) # uint8是无符号八位整型，表示范围是[0, 255]的整数
# img_res = cv2.imdecode(nparr, cv2.IMREAD_COLOR)
# img_res_BGR = cv2.cvtColor(img_res, cv2.COLOR_RGB2BGR)
# fig=plt.imshow(img_res_BGRabs)
'''__________________ 方法2：____________________'''
# img=plt.imread(img_name) 
# fig=plt.imshow(img)
'''__________________ 方法3：____________________'''
# img = cv2.imread(img_name)    # 必须是英文路径
# img_BGR = cv2.cvtColor(img,cv2.COLOR_RGB2BGR)
# fig=plt.imshow(img_BGR)

plt.show()

In [ ]:
# （五）json + execjs + datetime + requests + urllib.parse.urlencode 百度指数：王者荣耀
import time
import json
import execjs  # pip install PyExecJS
import datetime
import requests
from urllib.parse import urlencode
def get_data(keywords, startDate, endDate, area):
    """获取加密的参数数据"""
    # data_url = "http://index.baidu.com/api/SearchApi/index?area=0&word=[[%7B%22name%22:%22%E7%8E%8B%E8%80%85%E8%8D%A3%E8%80%80%22,%22wordType%22:1%7D]]&startDate=2020-10-01&endDate=2020-10-10"
    params = {'word': json.dumps([[{'name': keyword, 'wordType': 1}] for keyword in keywords]),
        'startDate': startDate,'endDate': endDate,'area': area}
    data_url = 'http://index.baidu.com/api/SearchApi/index?' + urlencode(params)
    print(data_url)
    headers = {# 复制登录后的cookie
        "Cookie": 'BAIDUID=105263D83A6CA7DAF6E92FA1078C647D:FG=1; PSTM=1602161735; BIDUPSID=324C2FF381B2F05DA0A9AF3A86143D86; H_WISE_SIDS=154758_153901_157882_156598_158204_157009_156817_156287_150775_154259_148867_155226_154606_153628_157264_158926_151532_150772_151015_156641_156387_153065_156515_127969_154412_154174_152982_158528_150345_146732_155791_131423_157706_154038_158283_107312_158055_154189_155344_155255_158024_157790_144966_156710_154213_157814_158717_158641_156847_157188_147552_159045_158367_158587_157697_154639_159093_154347_157472_159074_110085_157006; BDUSS=TloWTliQVlQRm9JbFd6bEdNY3dnOVJRY1AwUmQ0TTF6T1Foa3A4dWRaZjN4YjlmRVFBQUFBJCQAAAAAAAAAAAEAAACQf3g3s8K379LHY2Z5AAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAPc4mF~3OJhfO; __yjsv5_shitong=1.0_7_b134f444b7a7001ed7d7a761294f4d78117d_300_1604321777219_120.84.12.145_6ffe2a37; bdindexid=0s0i98k3dk5nsbd0f6ldihi5s1; Hm_lvt_d101ea4d2a5c67dab98251f0b5de24dc=1603811385,1603975247,1604321651,1604321977; Hm_lpvt_d101ea4d2a5c67dab98251f0b5de24dc=1604322212; RT="z=1&dm=baidu.com&si=36foifjisc8&ss=kh0jnn93&sl=a&tt=2ux3&bcn=https%3A%2F%2Ffclog.baidu.com%2Flog%2Fweirwood%3Ftype%3Dperf&ld=db8q',
        "Referer": "http://index.baidu.com/v2/main/index.html",
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/77.0.3865.90 Safari/537.36"
}
    # 获取data和uniqid
    res = requests.get(url=data_url, headers=headers).json()
    print(res)
    data = res["data"]["userIndexes"][0]["all"]["data"]
    uniqid = res["data"]["uniqid"]
    # 获取js函数中的参数t = "ev-fxk9T8V1lwAL6,51348+.9270-%"
    t_url = "http://index.baidu.com/Interface/ptbk?uniqid={}".format(uniqid)
    rep = requests.get(url=t_url, headers=headers).json()
    t = rep["data"]
    return {"data": data, "t": t}

def get_search_index(word, startDate, endDate, area):
    """获取最终数据"""
    word = word
    startDate = startDate
    endDate = endDate
    # 调用get_data获取data和uniqid
    res = get_data(word, startDate, endDate, area)
    e = res["data"]
    t = res["t"]
    print(e,t)
    # 读取js文件
    with open('parsing_data_function.js', encoding='utf-8') as f:
        js = f.read()
    # 通过compile命令转成一个js对象
    docjs = execjs.compile(js)
    # 调用function方法,得到指数数值
    res = docjs.call('decrypt', t, e)
    # print(res)
    return res
def get_date_list(begin_date, end_date):
    """获取时间列表"""
    dates = []
    dt = datetime.datetime.strptime(begin_date, "%Y-%m-%d")
    date = begin_date[:]
    while date <= end_date:
        dates.append(date)
        dt += datetime.timedelta(days=1)
        date = dt.strftime("%Y-%m-%d")
    return dates
def get_area():
#     areas = {"901": "山东", "902": "贵州", "903": "江西", "904": "重庆", "905": "内蒙古", "906": "湖北", 
#              "907": "辽宁", "908": "湖南", "909": "福建", "910": "上海", "911": "北京", "912": "广西", 
#              "913": "广东", "914": "四川", "915": "云南", "916": "江苏", "917": "浙江", "918": "青海",
#              "919": "宁夏", "920": "河北", "921": "黑龙江", "922": "吉林", "923": "天津", "924": "陕西",
#              "925": "甘肃", "926": "新疆", "927": "河南", "928": "安徽", "929": "山西", "930": "海南", 
#              "931": "台湾", "932": "西藏", "933": "香港", "934": "澳门"}
    areas = {"901": "山东"}
    for value in areas.keys():
        try:
            word = ['王者荣耀']
            time.sleep(1)
            startDate = '2020-10-01'
            endDate = '2020-10-10'
            area = value
            res = get_search_index(word, startDate, endDate, area)
            result = res.split(',')
            dates = get_date_list(startDate, endDate)
            for num, date in zip(result, dates):
                print(areas[value], num, date)
                with open('area.csv', 'a+', encoding='utf-8') as f:
                    f.write(areas[value] + ',' + str(num) + ',' + date + '\n')
        except:
            pass
def get_word():
    words = ['诸葛大力', '张伟', '胡一菲', '吕子乔', '陈美嘉', '赵海棠', '咖喱酱', '曾小贤', '秦羽墨']
    for word in words:
        try:
            time.sleep(2)
            startDate = '2020-10-01'
            endDate = '2020-10-10'
            area = 0
            res = get_search_index(word, startDate, endDate, area)
            result = res.split(',')
            dates = get_date_list(startDate, endDate)
            for num, date in zip(result, dates):
                print(word, num, date)
                with open('word.csv', 'a+', encoding='utf-8') as f:
                    f.write(word + ',' + str(num) + ',' + date + '\n')
        except:
            pass
get_area()
# get_word()

In [ ]:
# （六）requests+jsonfake_useragent ：搜狗壁纸 http://pic.sogou.com/pics/channel
import requests, json
from fake_useragent import UserAgent

class ShouGO(object):
    def __init__(self):
        ua = UserAgent(verify_ssl=False)
        for i in range(1, 50):
            self.headers = {'User-Agent': ua.random,}
    def Shou(self, category, length, path):
        n = length
        cate = category
        imgs = requests.get(
            'http://pic.sogou.com/pics/channel/getAllRecomPicByTag.jsp?category=' + cate + '&tag=%E5%85%A8%E9%83%A8&start=0&len=' + str(
                n))
        jd = json.loads(imgs.text)
        print(imgs.text)
#         jd = jd['all_items']
#         imgs_url = []
#         for j in jd:
#             imgs_url.append(j['pic_url'])
#         m = 0
#         for img_url in imgs_url:
#             # print(img_url)
#             print('***** ' + cate + str(m) + '.jpg *****' + '   Downloading...')
#             img = requests.get(url=img_url, headers=self.headers).content
#             with open(path + cate + str(m) + '.jpg', 'wb') as f:
#                 f.write(img)
#             m = m + 1
#         print('Download complete!')

    def main(self):
        self.Shou('汽车', 2, './壁纸2/')


if __name__ == '__main__':
    Siper = ShouGO()
    Siper.main()